In [1]:
import pandas as pd
import json

### Helper to convert colums to rows for creating Data Frame

In [2]:
def T(rows_or_cols): return list(zip(*rows_or_cols))

matrix = [
    (1, 2, 3),
    (4, 5, 6),
]

transpose = [
    (1, 4),
    (2, 5),
    (3, 6),
]

assert T(matrix) == transpose
assert T(T(matrix)) == matrix

### Convert a period number to hour, minute tuple depending on number of intervals

In [3]:
def period_to_hours_minutes(period, n_intervals=48):
    # 1 -> 0, 30
    # 2 -> 1, 0
    # 48 -> 24, 0
    assert 1 <= period <= period
    assert int(period) == period
    
    minutes = (1440 / n_intervals)  * period
    return int(minutes // 60), int(minutes % 60)
    
assert period_to_hours_minutes(1) == (0, 30)
assert period_to_hours_minutes(2) == (1, 0)
assert period_to_hours_minutes(3) == (1, 30)
assert period_to_hours_minutes(48) == (24, 0)

assert period_to_hours_minutes(1, 24) == (1, 0)
assert period_to_hours_minutes(4, 12) == (8, 0)
assert period_to_hours_minutes(24, 24) == (24, 0)
assert period_to_hours_minutes(12, 12) == (24, 0)

### Covert hours, minutes tuple to time

In [4]:
def hours_minutes_to_time(hm):
    # (hours, minutes) -> "hours:minutes:00"
    # (0, 30) -> "00:30:00"
    hours, minutes = hm
    return f"{hours:02}:{minutes:02}:00"
    
assert hours_minutes_to_time((0, 30)) == '00:30:00'
assert hours_minutes_to_time((5, 30)) == '05:30:00'
assert hours_minutes_to_time((23, 0)) == '23:00:00'

### Convert Period to time

In [5]:
def period_to_time(period): 
    return hours_minutes_to_time(period_to_hours_minutes(period))

assert period_to_time(1) == '00:30:00'
assert period_to_time(11) == '05:30:00'
assert period_to_time(48) == '24:00:00'

### Validate date

In [6]:
from datetime import timedelta, datetime

MIN_DATE = datetime(2020, 1, 1)



# Convenience method to get date from date time
def S(dt): return str(dt.date())

def AssertException(exc, f, *args, **kwargs):
    try:
        f(*args, **kwargs)
    except exc:
        return True
    
    return False


def validate_date(d):
    year, month, day = map(int, d.split("-"))
    date = datetime(year, month, day)
    
    if MIN_DATE - timedelta(days=1) >= date:
        raise ValueError(f'We only accept dates from {MIN_DATE}. You asked for {date}')
        
    if date >= datetime.now():
        raise ValueError(f'We do not accept future dates. You asked for {date}')
        
        
    return date

AE = AssertException
    
AE(ValueError, validate_date, "2019-1-1")
AE(ValueError, validate_date, "20")
AE(ValueError, validate_date, str(datetime.now().date() + timedelta(days=1)))

start = datetime(2020, 1, 5)

assert S(start) == "2020-01-05"
assert validate_date(S(start)) == start


### Get dates between 2 datetimes

In [7]:
def get_dates_between(start, end=datetime.now()):
    
    while start <= end:
        yield S(start)
        start += timedelta(days=1)
        
assert len(list(get_dates_between(start))) >= 172

end = datetime(2020, 1, 7)

assert list(get_dates_between(start, end)) == ['2020-01-05', '2020-01-06', '2020-01-07']

### Get dates between 2 dates in yyyy-mm--dd format

In [8]:
def get_dates_from_strings(start, end=None):
    
    start = validate_date(start)
    
    if not end:
        return get_dates_between(start)
    
    
    end = validate_date(end)
    
    if start > end: 
        start, end = end, start
        
    return get_dates_between(start, end)

assert list(get_dates_from_strings(S(start))) == list(get_dates_between(start))
assert list(get_dates_from_strings(S(start), S(end))) ==  list(get_dates_between(start, end))

### Create DF

In [9]:
_dates = list(get_dates_between(start, end))

def generate_df(dates, period=48):

    date, time, period = T(
        [
            (date, period_to_time(i), i)
            for date in dates
            for i in range(1, period+1)
        ]
    ) 


    return pd.DataFrame(data={
        "Date": date,
        "Time": time,
        "Period": period,
    })


generate_df(_dates, 3)

,Date,Time,Period
0,2020-01-05,00:30:00,1
1,2020-01-05,01:00:00,2
2,2020-01-05,01:30:00,3
3,2020-01-06,00:30:00,1
4,2020-01-06,01:00:00,2
5,2020-01-06,01:30:00,3
6,2020-01-07,00:30:00,1
7,2020-01-07,01:00:00,2
8,2020-01-07,01:30:00,3


### Convert DF to list of dicts

In [10]:
def df_to_json(df):
    data = [
        {
            key: row[key]
            for key in df
        }

        for _, row in df.iterrows()
    ]
    
    return data

df_to_json(df)

NameError: name 'df' is not defined

In [ ]:
def get_json_from_strings(start, end=None, period=48):
    
    dates = get_dates_from_strings(start, end)
    df = generate_df(dates, period)

    return df_to_json(df)

assert get_json_from_strings('2020-1-1', '2020-1-3', 4) == get_json_from_strings('2020-1-3', '2020-1-1', 4)

get_json_from_strings('2020-1-1', '2020-1-3', 4)

In [ ]:
get_json_from_strings('2020-1-1', period=4)

### Pandas Solution

In [ ]:
df = pd.DataFrame(data={"Date":_dates})
df = pd.DataFrame(df.values.repeat(48, axis=0), columns=df.columns)
df ["Period"] = (df.index % 48) + 1
df ['Time'] = df.Period.apply(period_to_time)
df[["Date", "Time", "Period"]]

In [ ]:
http://127.0.0.1:5000/from/2020-06 # malformed url
http://127.0.0.1:5000/from/2020-06-11 # from date to today
http://127.0.0.1:5000/from/2019-06-11 # too early
http://127.0.0.1:5000/from/2021-06-11 # no future dates
http://127.0.0.1:5000/from/2020-06-11/to/2020-06-13 # 2 dates
http://127.0.0.1:5000/from/2020-06-13/to/2020-06-11 # also works with dates reversed